In [34]:
import numpy as np
import pandas as pd
import random
from itertools import chain
from pathlib import Path


np.random.seed(0)

In [35]:
# root = Path.cwd() / 'stim_set_final_by_cat'
root = Path('/Users/mn/Desktop/g_images/stim_set_final_by_cat')
searchterm_paths = sorted([f for f in root.iterdir() if f.is_dir()])
search_terms = [p.name for p in searchterm_paths]

assert len(search_terms) == 24
print(search_terms)

['airplane', 'bat', 'boat', 'broccoli', 'cat', 'cow', 'elephant', 'face child', 'face female', 'face male', 'fish', 'flower', 'garden tools', 'gymnast', 'hammer', 'hand tools', 'houseplant', 'monkey', 'pineapple', 'surfer', 'tennis player', 'tomato', 'train', 'tree']


In [36]:
# copy all images to one folder

# import shutil
# out = '/Users/mn/Desktop/g_images/stim_set_final_dedup'
# jpegs = [r for r in root.glob('*/*.jpg')]

# for j in jpegs:
#     shutil.copy(j, out)

In [37]:
# class - anim x inanim

classification = {'animate': ['bat', 'cat', 'cow', 'elephant', 'face child', 'face female', 
                        'face male', 'fish', 'gymnast', 'monkey', 'surfer', 'tennis player']}
classification['inanimate'] = [cat for cat in search_terms if cat not in classification['animate']] # rest

assert len(classification['animate']) == len(classification['inanimate'])


# object category

object_categories = {
    "body (human)":     ['gymnast', 'surfer', 'tennis player'],
    "face (human)":     ['face child', 'face female', 'face male'],
    "body (animal)":    ['bat', 'elephant', 'fish'],
    "face (animal)":    ['cow', 'cat', 'monkey'],
    "plant":            ['flower', 'houseplant', 'tree'],
    "fruit / vegetable":['broccoli', 'tomato', 'pineapple'],
    "tool":             ['garden tools', 'hammer', 'hand tools'],
    "vehicle":          ['airplane', 'boat', 'train'],
}

assert len(list(chain(*object_categories.values()))) == 24

In [38]:
image_dict = {}

for i, category in enumerate(search_terms):
    image_dict[category] = sorted([str(imgpath).split("/")[-1] for imgpath in searchterm_paths[i].glob('*jpg') 
                                        if str(imgpath).split("/")[-1][:3] != 'dis'], 
                                        key=lambda x: int(''.join(filter(str.isdigit, str(x))))
                                        )

assert all([len(image_dict[k]) >= 30 for k in image_dict.keys()])

In [39]:
# overall - 120 imgs / 8 categories = 15 imgs / category
from copy import deepcopy
images_by_categories = deepcopy(object_categories)

for category in images_by_categories:
    for i, search_term in enumerate(images_by_categories[category]):
        images_by_categories[category][i] = image_dict[search_term]

In [40]:
# convert to df for easier manipulation

object_categories_astuples = []

for category in object_categories:
    for search_term in object_categories[category]:
        object_categories_astuples.append((category, search_term))

columns = pd.MultiIndex.from_tuples(sorted(object_categories_astuples))

images_df = pd.DataFrame.from_dict(image_dict, orient='index').T.dropna(how='any')
images_df = images_df[[c[1] for c in columns]]
images_df.columns = columns

assert len(images_df.index) == 30

#TODO: save df to csv
images_df.to_csv('images.csv', index=False)
images_df.head()

body (animal)                                 body (human)                 \
            bat        elephant         fish         gymnast         surfer   
0     bat_0.jpg  elephant_0.jpg   fish_2.jpg   gymnast_0.jpg   surfer_0.jpg   
1     bat_3.jpg  elephant_1.jpg   fish_3.jpg   gymnast_9.jpg   surfer_2.jpg   
2     bat_5.jpg  elephant_2.jpg   fish_8.jpg  gymnast_15.jpg   surfer_6.jpg   
3     bat_6.jpg  elephant_5.jpg  fish_14.jpg  gymnast_25.jpg  surfer_12.jpg   
4     bat_7.jpg  elephant_6.jpg  fish_15.jpg  gymnast_28.jpg  surfer_13.jpg   

                        face (animal)                            \
          tennis player           cat        cow         monkey   
0   tennis player_6.jpg     cat_0.jpg  cow_0.jpg   monkey_9.jpg   
1  tennis player_11.jpg     cat_2.jpg  cow_2.jpg  monkey_43.jpg   
2  tennis_player_20.jpg    cat_4b.jpg  cow_3.jpg  monkey_49.jpg   
3  tennis player_22.jpg     cat_4.jpg  cow_4.jpg  monkey_67.jpg   
4  tennis player_23.jpg     cat_8.jpg  cow_5.jpg  monkey_97.jpg   

        face (human)  ... fruit / vegetable          plant                     \
          face child  ...            tomato         flower         houseplant   
0   face_new_b_0.jpg  ...      tomato_1.jpg   flower_0.jpg   houseplant_4.jpg   
1  baby's face_4.jpg  ...      tomato_2.jpg  flower_12.jpg   houseplant_7.jpg   
2   face_new_b_5.jpg  ...      tomato_3.jpg  flower_14.jpg   houseplant_8.jpg   
3  baby's face_8.jpg  ...     tomato_15.jpg  flower_18.jpg  houseplant_15.jpg   
4   face_new_c_9.jpg  ...     tomato_20.jpg  flower_22.jpg  houseplant_22.jpg   

                        tool                                     \
           tree garden tools          hammer         hand tools   
0    tree_1.jpg   rake_0.jpg    hammer_1.jpg   hand tools_1.jpg   
1    tree_4.jpg   rake_6.jpg    hammer_2.jpg   hand tools_3.jpg   
2    tree_8.jpg   rake_8.jpg    hammer_3.jpg  hand tools_11.jpg   
3   tree_10.jpg  rake_10.jpg    hammer_9.jpg      wrench_18.jpg   
4  tree_10b.jpg  rake_11.jpg  hammer_2_0.jpg  hand tools_20.jpg   

           vehicle                             
          airplane         boat         train  
0   airplane_0.jpg   boat_2.jpg   train_0.jpg  
1      plane_1.jpg   boat_3.jpg   train_2.jpg  
2      plane_5.jpg  boat_10.jpg   train_4.jpg  
3   airplane_8.jpg  boat_11.jpg   train_6.jpg  
4  airplane_12.jpg  boat_12.jpg  train_11.jpg  

[5 rows x 24 columns]

In [41]:
# per one stim set - 120 imgs / 8 categories = 15 imgs / category
# => split into 6 sets every 5 rows

stimA = images_df[:5]
stimB = images_df[5:10]
stimC = images_df[10:15]
stimD = images_df[15:20]
stimE = images_df[20:25]
stimF = images_df[25:30]

In [42]:
def reshape_multiindex(stim_df: pd.DataFrame) -> pd.DataFrame:
    stim_df.columns = stim_df.columns.get_level_values(0)
    s = stim_df.columns.to_series()
    stim_df.columns = [stim_df.columns, s.groupby(s).cumcount()]
    stim_df = (stim_df
                    .stack()
                    # .sample(frac=1, axis=0)
                    .apply(np.random.permutation, axis=0)
                    .reset_index(drop=True)
                )
    return stim_df


def list_blocks(stim_df_reshaped: pd.DataFrame) -> list[list[str]]:
    # split last 3 rows by animate/inanimate
    animate = "body (animal),body (human),face (animal),face (human)".split(',')
    inanimate = "fruit / vegetable,plant,tool,vehicle".split(',')
    rem = stim_df_reshaped.iloc[-3:]
    rem_anim = list(rem[animate].iloc[0]) + list(rem[animate].iloc[1]) + list(rem[animate].iloc[2])
    rem_inanim = list(rem[inanimate].iloc[0]) + list(rem[inanimate].iloc[1]) + list(rem[inanimate].iloc[2])
    
    # split into blocks -> 10 imgs per block (5 anim, 5 inanim), 1 per category (with repeats)
    blocks = []
    for i in range(12):
        blocks.append(
            list(stim_df_reshaped.iloc[i]) + [rem_anim[i]] + [rem_inanim[i]]
        )
    # randomize order
    for block in blocks: np.random.shuffle(block)
    
    return blocks


def from_stim_set_blocks(stim_df: pd.DataFrame) -> list[list[str]]:
    reshaped_df = reshape_multiindex(stim_df)
    blocks = list_blocks(reshaped_df)

    return blocks

In [43]:
def get_key(item: str, d: dict) -> str:
    categories = filter(lambda category: item in d[category], d)
    return categories.__next__()

def assign_category(image: str) -> tuple[str, str]:
    term = get_key(image, image_dict)
    category = get_key(term, object_categories)

    return (image, category)

In [44]:
# blocks = from_stim_set_blocks(stimA)

In [45]:
def create_blocks_final(blocks: list[list[str]]) -> list[list[str]]:
    for block in blocks:
        for i, image in enumerate(block):
            block[i] = assign_category(image)

    prefix_add = lambda t: ("dis_" + t[0], t[1])
    prefix_strip = lambda s: s if s[:3] != 'dis' else s[4:]

    for block in blocks:
        post_imgs = deepcopy(block)
        dis_imgs = [prefix_add(item) for item in block]
        np.random.shuffle(post_imgs)
        np.random.shuffle(dis_imgs)
        block.extend(dis_imgs + post_imgs)

    for block in blocks:
        for i in range(len(block) - 2):
            if prefix_strip(block[i][0]) == prefix_strip(block[i+1][0]):
                # print(block[i][0], block[i+1][0])
                block[i+1], block[i+2] = block[i+2], block[i+1]

    return blocks

In [46]:
# blocks = from_stim_set_blocks(stimA)

# # map images to category (i.e. reverse dict lookup: img -> search_term -> category)
# for block in blocks:
#     for i, image in enumerate(block):
#         block[i] = assign_category(image)

# prefix_add = lambda x: ("dis_" + x[0], x[1])

# for block in blocks:
#     post_imgs = deepcopy(block)
#     dis_imgs = [prefix_add(item) for item in block]
#     np.random.shuffle(post_imgs)
#     np.random.shuffle(dis_imgs)
#     block.extend(dis_imgs + post_imgs)

In [47]:
def create_stim_spreadsheet(blocks: list[list[str]]) -> pd.DataFrame:

    df = pd.DataFrame(chain(*blocks), columns=['image', 'answer'])

    # f = lambda l, r, t: np.tile(np.repeat(np.array(np.arange(1, l)), r), t)
    display = pd.Series(['pre', 'disambig', 'post']).repeat(10).reset_index(drop=True)
    randomise_trials = pd.Series(range(1,37)).repeat(10).reset_index(drop=True)
    randomise_blocks = pd.Series(range(1,13)).repeat(30).reset_index(drop=True)
    display_col = pd.concat([display] * 12).reset_index(drop=True)
    randomise_trials_col = pd.concat([randomise_trials] * 12).reset_index(drop=True)

    df['display'] = display_col
    df['progress'] = 1
    df['randomise_trials'] = randomise_trials_col
    df['randomise_blocks'] = randomise_blocks
    df['ShowProgressBar'] = np.nan
    df.astype({'randomise_blocks': 'int32', 'randomise_trials': 'int32', 'progress': 'int32'})

    # add block separators
    idxs = np.arange(-0.5, 330, 30)

    for i, idx in enumerate(idxs):
        df.loc[idx, 'display'] = 'block'
        df.loc[idx, 'text'] = f'Block {i+1} of 12'
    df = (df
          .sort_index()
          .reset_index(drop=True)
          .astype({'randomise_blocks': 'Int64', 'randomise_trials': 'Int64', 'progress': 'Int64'})
        )

    return df[['randomise_blocks', 'display', 'image', 
              'answer', 'text', 'progress', 'ShowProgressBar']]


In [48]:
blocks_short = [from_stim_set_blocks(stim) for stim in (stimA, stimB, stimC, stimD, stimE, stimF)]
blocks_long = [create_blocks_final(blocks) for blocks in blocks_short]
spreadsheets = [create_stim_spreadsheet(blocks) for blocks in blocks_long]

# for i, sheet in enumerate(spreadsheets):
#     sheet.to_csv(f'spreadsheet{i+1}.csv', index=False)

In [49]:
f = lambda x: x if x[:3] != 'dis' else x[4:]

for n, stim in enumerate(blocks_long):
    for m, block in enumerate(stim):
        for i in range(len(block) - 2):
            if f(block[i][0]) == f(block[i+1][0]):
                print(n, m)
                print(block[i][0], block[i+1][0])

In [50]:
# idxs = np.arange(-0.5, 330, 30)

# for i, idx in enumerate(idxs):
#     df.loc[idx, 'display'] = 'block'
#     df.loc[idx, 'text'] = f'Block {i+1} of 12'
# df = df.sort_index().reset_index(drop=True)
# df.head(90)